## Bulletines Cleaning

I used this notebook to clean raw data previously scraped containing information from the monthly bulletines published by the Italian Ministry of Justice.

In [1]:
import pandas as pd
import numpy as np
import re

### Basic cleaning

- Converting numbers into integers (note that the `.` in Italy is being used as a decimal separator)
- Removing `Totals` (we'll calculate them when needed)
- Fixing empty values to numeric (from NaN to 0)
- Converting to integer

In [ ]:

# Read CSV with thousands separator specified
df = pd.read_csv("../outputs/raw/bulletines_raw.csv", thousands=".")

# Remove rows where "Regione di detenzione" is "Totale"
df = df[df["Regione di detenzione"] != "Totale"]

# Replace empty strings and similar with NaN
df.replace(["", " ", "NaN", "nan"], np.nan, inplace=True)

# Fill NaN values with 0
df.fillna(0, inplace=True)

# Convert columns to integer type
df['Detenuti presenti - stranieri'] = df['Detenuti presenti - stranieri'].astype(int)

# Remove dots from numeric strings in the specified column
df['Detenuti presenti - totale'] = df['Detenuti presenti - totale'].str.replace(".", "")

# Show a random sample of 5 rows
df.sample(5)


In [ ]:
df.dtypes

In [ ]:
# Step 1: Replace dots in the 'Detenuti presenti - totale' column, then convert to numeric
df['Detenuti presenti - totale'] = df['Detenuti presenti - totale'].str.replace(".", "").astype(float)
# Step 2: Fill any remaining NaN values with 0
df['Detenuti presenti - totale'].fillna(0, inplace=True)
# Step 3: Convert the cleaned column to integers
df['Detenuti presenti - totale'] = df['Detenuti presenti - totale'].astype(int)
df['Detenuti presenti - donne'] = df['Detenuti presenti - donne'].astype(int)


In [ ]:
df.sort_values('Detenuti presenti - stranieri', ascending=False).head(15)

In [ ]:
df.dtypes

In [7]:
# Convert the 'Ultimo aggiornamento' column to datetime format
df['Ultimo aggiornamento'] = pd.to_datetime(df['Ultimo aggiornamento'])


In [ ]:
df.dtypes

### Fixing names

A problem we are encountering are the many different names each detention center has been registered with over the years. Here below we use [thefuzz](https://github.com/seatgeek/thefuzz) to do an initial fuzzy matching, and then fix the remaining ones manually. As a reference, we'll use the information we have scraped in a different notebook about all the detention centers currently operating in Italy.

In [9]:
df['Istituto'] = df['Istituto'].str.strip()
df['Istituto'] = df['Istituto'].str.replace(r"\s*-", "", regex=True)  # Removes any whitespace followed by a dash


In [ ]:
sorted(df['Istituto'].unique())

In [11]:
from thefuzz import process

In [ ]:
# Import standard names
df_standard_names = pd.read_csv('../outputs/clean/institutes_info.csv')
standard_names = df_standard_names['nome_istituto'].tolist()
standard_names

In [ ]:
df_copy = df
df_copy.head(4)

In [14]:
def standardize_name(name):
    match, score = process.extractOne(name, standard_names)
    return match if score >= 90 else name  # Adjust score threshold as needed
# Apply fuzzy matching to standardize names
df_copy['Istituto'] = df_copy['Istituto'].apply(standardize_name)

In [ ]:
sorted(df_copy['Istituto'].unique())

Because the names that did not match are uppercase, we can identify them easily and proceed with a mapping.

In [ ]:
sorted(df_copy[df_copy['Istituto'].str.isupper()]['Istituto'].unique())

In [ ]:
institute_mapping = {
    'BRESCIA "N. FISCHIONE" CANTON MONBELLO': 'Brescia Canton Monbello',
    'BRESCIA "NERIO FISCHIONE" CANTON MONBELLO': 'Brescia Canton Monbello',
    'CAGLIARI "E.SCALAS"': 'Cagliari Uta',
    'CAGLIARI "ETTORE SCALAS"': 'Cagliari Uta',
    'CAMERINO': 'Camerino',
    'CIVITAVECCHIA "N.C."': 'Civitavecchia Nuovo Complesso',
    "FORLI'": 'Forlì',
    'MILANO "F. DI CATALDO" SAN VITTORE': 'Milano San Vittore',
    'MILANO "FRANCESCO DI CATALDO" SAN VITTORE': 'Milano San Vittore',
    'NAPOLI "G. SALVIA" POGGIOREALE': 'Napoli Poggioreale',
    'NAPOLI "GIUSEPPE SALVIA" POGGIOREALE': 'Napoli Poggioreale',
    'NAPOLI "P. MANDATO" SECONDIGLIANO': 'Napoli Secondigliano',
    'NAPOLI "PASQUALE MANDATO" SECONDIGLIANO': 'Napoli Secondigliano',
    'PALERMO "A. LORUSSO" PAGLIARELLI': 'Palermo Pagliarelli',
    'PALERMO "ANTONIO LORUSSO" PAGLIARELLI': 'Palermo Pagliarelli',
    'PALERMO "C. DI BONA" UCCIARDONE': 'Palermo Ucciardone',
    'PALERMO "CALOGERO DI BONA" UCCIARDONE': 'Palermo Ucciardone',
    'ROMA "G. STEFANINI" REBIBBIA FEMMINILE': 'Roma Rebibbia Femminile',
    'ROMA "GERMANA STEFANINI" REBIBBIA FEMMINILE': 'Roma Rebibbia Femminile',
    'ROMA "R. CINOTTI" REBIBBIA N.C.1': 'Roma Rebibbia',
    'ROMA "RAFFAELE CINOTTI" REBIBBIA N.C.1': 'Roma Rebibbia',
    'ROMA "REBIBBIA TERZA CASA"': 'Roma Rebibbia III Casa',
    'SAN REMO "N.C."': 'Sanremo',
}

df_copy['Istituto'] = df_copy['Istituto'].replace(institute_mapping)

# Check the updated unique values
df_copy['Istituto'].unique()

In [ ]:
df_copy.sample(5)

In [19]:
# # remove duplicate data from Dec 2021
# df_copy = df_copy[df_copy['ID'] != 'SST360932']

In [20]:
df_copy.to_csv('../outputs/clean/bulletines.csv', index=False, encoding="UTF-8-sig")